# Discretized 1D Kohn-Sham Equation

## Problem Description
In this part, we consider the singleparticle Hamiltonian arising from discretizing an 1D Kohn-Sham equation in electronic structure
calculations,

$$
\begin{aligned}
    \min_{X \in \mathbb{R}^{n\times p}}\quad &\frac{1}{2} \mathrm{tr}\left( X^\top LX \right) + \frac{\alpha}{4} \rho^\top L^{-1} \rho\\
    \text{s. t.} \quad &X^\top X = I_p,
\end{aligned}
$$

where $\rho := \mathrm{Diag}(XX^\top)$, $L$ is a tri-diagonal matrix with $2$ on its diagonal and $-1$ on its subdiagonal, and $\alpha > 0$ is a parameter. Such problems have become standard testing problems for investigating the convergence of self-consistent field methods due to its simplicity. Clearly, these problems are smooth optimization problems on the Stiefel manifold, and we show how to solve these problems by our proposed ExPen penalty function.

In [1]:
# Add essential packages
import numpy as np
import scipy as sp
from stiefel import stiefel
import time

# Add essential solvers
from scipy.optimize import fmin_bfgs, fmin_cg, fmin_l_bfgs_b, fmin_ncg
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [2]:
# Set parameters
n = 1000
p = 20
alpha = 1

In [3]:
# Generate manifold
M = stiefel(n,p)
A = M.A
C = M.C 
JA = M.JA_exact
Xinit = M.Init_point()
# Since those solvers only receives vectors, we need to reshape the variables in ecah iteration.
def v2m(x):
    return np.reshape(x, (n,p))

def m2v(X):
    return X.flatten()

L = diags(np.array([-1, 2, -1]), np.array([1, 0, -1]), shape = (n,n)).tocsc()

In [4]:
# Define obnjective function and their corresponding ExPen
def obj_fun(X):
    LX = L@X
    rho = np.sum(X * X, 1)
    Lrho = spsolve(L, rho)
    fval = 0.5*np.sum(X* LX) + (alpha /4) * np.sum(rho * Lrho)
    return fval


def obj_grad(X):
    LX = L@X
    rho = np.sum(X * X, 1)
    Lrho = spsolve(L, rho)
    grad = LX + alpha * Lrho[: ,np.newaxis] * X
    return   grad    



beta = 0.5 * np.linalg.norm(Xinit.T @obj_grad(Xinit),'fro')

def obj_fun_expen(y):
    Z = v2m(y)
    X = M.A(Z)
    LX = L@X
    rho = np.sum(X * X, 1)
    Lrho = spsolve(L, rho)
    fval = 0.5*np.sum(X* LX) + (alpha /4) * np.sum(rho * Lrho) + beta/4 * (np.linalg.norm(Z.T @ Z - np.eye(p),'fro') ** 2)
    return fval

def obj_grad_expen(y):
    Z = v2m(y)
    X = M.A(Z)
    LX = L@X
    rho = np.sum(X * X, 1)
    Lrho = spsolve(L, rho)
    grad = JA(Z, LX + alpha * Lrho[: ,np.newaxis] * X) + beta * (Z @(Z.T @ Z ) - Z)
    return m2v(grad)

In [5]:
# Call the solvers from scipy.optimize to minimize ExPen
t_start = time.time()
out_msg = sp.optimize.minimize(obj_fun_expen, Xinit.flatten(),method='L-BFGS-B',jac = obj_grad_expen, options={'gtol': 1e-3, 'disp': True})
t_end = time.time()

In [6]:
out_msg

      fun: 210.70870327496834
 hess_inv: <20000x20000 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.68108531e-05,  1.39671634e-04, -1.51495270e-04, ...,
       -7.70268868e-04,  2.38533140e-03, -8.19179552e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 977
      nit: 892
     njev: 977
   status: 0
  success: True
        x: array([-0.12841976, -0.14660123,  0.24613424, ...,  0.07782798,
        0.0699216 , -0.0326975 ])